AWS versus CESM
======

Import custom modules for working with model and AWS data
----

In [1]:
from netCDF4 import Dataset
import pandas as pd
import numpy as np
from IPython.display import HTML

from model import Model
from GCNet import GCNet

Read/create CESM LE metadata
----

In [2]:
# M = Model("cesmle_geog.nc")
M = Model("cesmle_geog_sub.nc")
M.loadMeta()
# M.info()
# dir(M)
# M.getLat2d()
# M.createMeta()

minLon = -70 + 360
maxLon = -15 + 360
minLat = 58
maxLat = 86.5
M.setDomain( (minLat, maxLat, minLon, maxLon) )

1d dataset


Load AWS metadata
----

In [3]:
A = GCNet("site_info.nc")
A.loadMeta()
nSites = A.getNSites()

In [4]:
qcSub = A.loadMeta('qc_AirT1_1996-2005')
qcFull = A.loadMeta('qc_AirT1_1996-2014')
# print qcSub
# print qcFull

Find closest grid points to each AWS
-----

In [5]:
C = []
for awsIX in (range(nSites)):
# for awsIX in ((0,1)):
    """ Basic AWS info """
    A.info(awsIX)
    awsName = A.getName( awsIX )
    awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))
    awsElev = A.getElev( awsIX )
    
    """ Find closest points """
    closest = M.closestPoints( awsLatLon )
    C.append( closest )
    
    """ Print info on closest points """
    print "Closest model points"
    P = []
    for d in closest:
        px = d['ixLon']
        py = d['ixLat']
        pdist = d['distance']
        plat  = M.getLat2d(py,px)
        plon  = M.getLon2d(py,px)
        pelev = M.getElev(py,px)
        perr = pelev - awsElev
        L = (py, px, pdist, plat, plon, pelev, perr)
        print "G(%d,%d): M: D = %.2f km, lat/lon/elev = %.2f/%.2f/%.2f m, elev_err = %.2f m" \
                     % L
        P.append( L ) 
    print "-"*80


Site 1: Swiss_Camp
Lat, Lon: 69.5732 310.7048
Elevation: 1176.0
Closest model points
G(11,17): M: D = 40.18 km, lat/lon/elev = 69.27/311.25/1066.12 m, elev_err = -109.88 m
G(11,16): M: D = 43.81 km, lat/lon/elev = 69.27/310.00/564.24 m, elev_err = -611.76 m
G(12,17): M: D = 73.77 km, lat/lon/elev = 70.21/311.25/1272.50 m, elev_err = 96.50 m
G(12,16): M: D = 75.72 km, lat/lon/elev = 70.21/310.00/777.15 m, elev_err = -398.85 m
--------------------------------------------------------------------------------
Site 2: Crawford_Point1
Lat, Lon: 69.8819 313.0264
Elevation: 2022.0
Closest model points
G(12,18): M: D = 41.55 km, lat/lon/elev = 70.21/312.50/1786.00 m, elev_err = -236.00 m
G(12,19): M: D = 45.62 km, lat/lon/elev = 70.21/313.75/2155.25 m, elev_err = 133.25 m
G(11,18): M: D = 71.38 km, lat/lon/elev = 69.27/312.50/1618.56 m, elev_err = -403.44 m
G(11,19): M: D = 73.93 km, lat/lon/elev = 69.27/313.75/2035.88 m, elev_err = 13.88 m
-------------------------------------------------------

Closest model points
G(11,16): M: D = 17.16 km, lat/lon/elev = 69.27/310.00/564.24 m, elev_err = -3.76 m
G(11,15): M: D = 49.79 km, lat/lon/elev = 69.27/308.75/307.74 m, elev_err = -260.26 m
G(12,16): M: D = 87.83 km, lat/lon/elev = 70.21/310.00/777.15 m, elev_err = 209.15 m
G(12,15): M: D = 99.01 km, lat/lon/elev = 70.21/308.75/463.62 m, elev_err = -104.38 m
--------------------------------------------------------------------------------
Site 18: KULU
Lat, Lon: 65.7584 320.3982
Elevation: 878.0
Closest model points
G(7,24): M: D = 34.31 km, lat/lon/elev = 65.50/320.00/655.89 m, elev_err = -222.11 m
G(7,25): M: D = 48.70 km, lat/lon/elev = 65.50/321.25/325.04 m, elev_err = -552.96 m
G(8,24): M: D = 77.88 km, lat/lon/elev = 66.44/320.00/1643.77 m, elev_err = 765.77 m
G(8,25): M: D = 84.95 km, lat/lon/elev = 66.44/321.25/1362.82 m, elev_err = 484.82 m
--------------------------------------------------------------------------------
Site 19: JAR3
Lat, Lon: 69.3954 309.6896
Elevation: 323.0

Interpolate model elevation to AWS location
====

In [6]:
df = pd.DataFrame(columns=['Site #','Site name','Site lat','Site lon','Site elev','ElevCorr','ElevErr',
                          'QC Full', 'QC Overlap','Dry T Adj','Moist T Adj'],index=range(nSites))
for awsIX in (range(nSites)):
# for awsIX in ((0,1)):
    """ Basic AWS info """
    A.info(awsIX)
    awsName = A.getName( awsIX )
    awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))
    awsElev = A.getElev( awsIX )

    """ Perform the interpolation on elev from model metadata """
    closest = C[awsIX]
    Z = [ M.getElev( d[0], d[1] ) for d in zip( closest['ixLat'], closest['ixLon'] )]
    Znew = M.interpolate( Z, closest, awsLatLon )
    perr = Znew - awsElev
    print "\nNew elev = %0.2f, elev_err = %0.2f" % ( Znew, perr )
    dryAdAdj = perr/1000.*9.8
    mstAdAdj = perr/1000.*6.0
    row = ( awsIX+1, awsName, "%.2f" % awsLatLon[0], "%.2f" % awsLatLon[1], awsElev, \
            "%.2f" % Znew, "%.2f" % perr, \
            "%.2f" % qcFull[awsIX], "%.2f" % qcSub[awsIX], "%.2f" % dryAdAdj, "%.2f" % mstAdAdj )
    df.loc[awsIX] = row
    print "-"*80

Site 1: Swiss_Camp
Lat, Lon: 69.5732 310.7048
Elevation: 1176.0

New elev = 912.09, elev_err = -263.91
--------------------------------------------------------------------------------
Site 2: Crawford_Point1
Lat, Lon: 69.8819 313.0264
Elevation: 2022.0

New elev = 1896.44, elev_err = -125.56
--------------------------------------------------------------------------------
Site 3: NASA-U
Lat, Lon: 73.8333 310.5047
Elevation: 2334.0

New elev = 2130.69, elev_err = -203.31
--------------------------------------------------------------------------------
Site 4: GITS
Lat, Lon: 77.1433 298.905
Elevation: 1869.0

New elev = 984.18, elev_err = -884.82
--------------------------------------------------------------------------------
Site 5: Humboldt
Lat, Lon: 78.5266 303.1695
Elevation: 1995.0

New elev = 1762.56, elev_err = -232.44
--------------------------------------------------------------------------------
Site 6: Summit
Lat, Lon: 72.5794 321.4958
Elevation: 3199.0

New elev = 3068.80, elev

In [7]:
df

,Site #,Site name,Site lat,Site lon,Site elev,ElevCorr,ElevErr,QC Full,QC Overlap,Dry T Adj,Moist T Adj
0,1,Swiss Camp,69.57,310.70,1176,912.09,-263.91,69.25,75.07,-2.59,-1.58
1,2,Crawford Point1,69.88,313.03,2022,1896.44,-125.56,61.92,73.03,-1.23,-0.75
2,3,NASA-U,73.83,310.50,2334,2130.69,-203.31,51.73,35.75,-1.99,-1.22
3,4,GITS,77.14,298.90,1869,984.18,-884.82,23.73,19.65,-8.67,-5.31
4,5,Humboldt,78.53,303.17,1995,1762.56,-232.44,55.30,65.07,-2.28,-1.39
5,6,Summit,72.58,321.50,3199,3068.80,-130.20,76.64,70.59,-1.28,-0.78
6,7,Tunu-N,78.02,326.01,2052,2114.82,62.82,80.45,81.17,0.62,0.38
7,8,DYE-2,66.48,313.72,2099,2104.91,5.91,87.17,93.48,0.06,0.04
8,9,JAR1,69.50,310.32,932,742.77,-189.23,77.37,78.08,-1.85,-1.14
9,10,Saddle,66.00,315.50,2467,2348.23,-118.77,77.54,86.97,-1.16,-0.71


In [8]:
# save the dataframe of AWS info
df.to_pickle('cesmle_site_info.pkl' )

In [9]:
# print a DataFrame to HTML file (then print that to PDF in a browser!)
hout = HTML(df.to_html())
hfile = open("aws_table.htm","w")
hfile.write(hout.data)
hfile.close()